In [2]:
import sys
sys.path.insert(0, "../src")
from pathlib import Path

import pandas as pd

import constants
from gen.util import read_data, write_jsonl

In [3]:
LABEL2LABEL = {"SUPPORTS": "SUPPORT", "REFUTES": "CONTRADICT"}

In [6]:
def prepare_scifact_doc_submission(pred_p: Path):
    predictions = read_data(pred_p)
    submission = []
    
    for doc in predictions:
        sub_doc = {
            "id": int(doc["claim_id"].split("|")[1]),
            "evidence": {}
        }
        if doc["predicted_label"] != constants.LOOKUP["label"]["nei"]:
            for evidence in doc["predicted_evidence"]:
                if evidence[0] in sub_doc["evidence"]:
                    sub_doc["evidence"][evidence[0]]["sentences"].append(evidence[1])
                else:
                    sub_doc["evidence"][evidence[0]] = {
                        "sentences": [evidence[1]],
                        "label": LABEL2LABEL[doc["predicted_label"]]
                    }
        submission.append(sub_doc)
    
    return submission

def prepare_scifact_sent_submission():
    pass

In [7]:
tmp = prepare_scifact_doc_submission(Path("/users/k21190024/study/fact-check-transfer-learning/scratch/thesis/predictions/doc/scifact_test/fever-climatefeverpure-xlnet-base-cased.test.jsonl"))

In [11]:
tmp[11]

{'id': 84,
 'evidence': {'7521113': {'sentences': [4], 'label': 'SUPPORT'},
  '12827098': {'sentences': [5, 0], 'label': 'SUPPORT'},
  '22406695': {'sentences': [1], 'label': 'SUPPORT'},
  '12966719': {'sentences': [2], 'label': 'SUPPORT'}}}